<a href="https://colab.research.google.com/github/CarolineAndradeR/Atividade_nao_avaliativas_FIAP/blob/main/atividade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Modulos
import numpy as np
import tensorflow as tf
import pandas as pd

# Definição dos tamanhos
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE = 1000

# Importação dos dados
dados_review = pd.read_csv('/content/drive/MyDrive/review_clientes_melhores_compras.csv')
dados_review.head(10)

,review_id,review_cliente,sentimento
0,e64fb393e7b32834bb789ff8bb30750e,receber bem antar prazo estipular,1
1,f7c4243c7fe1938f181bec41a392bdeb,parabens melhores compras adorar comprar inter...,1
2,8670d52e15e00043ae7de4c01cc2fe06,recomendar aparelhar eficiente site marcar apa...,1
3,4b49719c8a200003f700d3d986ea1a19,pouco travar valor ta bom,1
4,3948b09f7c818e2d86c9a546758b2335,super recomendar vendedor confiavel produto ok...,1
5,9314d6f9799f5bfba510cc7bcd468c01,gostar saber sempre receber comprar agora decp...,0
6,373cbeecea8286a2b66c97b1b157ec46,nao chegar produto mau,0
7,d21bbc789670eab777d27372ab9094cc,otimo loja noto,1
8,0e0190b9db53b689b285d3f3916f8441,obrigar atencao amim dispensar,1
9,fe3db7c069d694bab50cc43463f91608,comprar realizar facilmente entregar efetuada ...,1


In [13]:
# Divisão dos dados em treino e teste
traino_dados_review = dados_review['sentimento']
teste_dados_review = dados_review['review_cliente']

In [18]:
# Criando objetos tf.data.Dataset a partir das colunas de treinamento e teste
traino_dados_review = tf.data.Dataset.from_tensor_slices((traino_dados_review, traino_dados_review))
teste_dados_review = tf.data.Dataset.from_tensor_slices((teste_dados_review, teste_dados_review))

# Embaralhando e agrupando os dados adequadamente
traino_dados_review = traino_dados_review.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
teste_dados_review = teste_dados_review.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [19]:
# Realizando o pre-processamento
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(traino_dados_review.map(lambda text, label: text))

UnimplementedError: ignored

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
BUFFER_SIZE = 10000
BATCH_SIZE = 64
VOCAB_SIZE=1000
# Baixando e carregando o conjunto de dados
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']
# Dividindo e embaralhando os dados
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# Realizando o pre-processamento
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))
# Criando o modelo
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
# Compilando o model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
# Treinando o modelo
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)
# Validando o modelo
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))
# Realizando uma predição
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions)